In [1]:
%maven org.nd4j:nd4j-cuda-9.0:1.0.0-beta
%maven org.deeplearning4j:deeplearning4j-cuda-9.0:1.0.0-beta
%maven org.datavec:datavec-api:1.0.0-beta

//%maven org.nd4j:nd4j-native-platform:1.0.0-beta
//%maven org.deeplearning4j:deeplearning4j-core:1.0.0-beta
//%maven org.datavec:datavec-api:1.0.0-beta

In [2]:
import org.deeplearning4j.datasets.iterator.*;
import org.deeplearning4j.datasets.iterator.impl.*;
import org.deeplearning4j.nn.api.*;
import org.deeplearning4j.nn.multilayer.*;
import org.deeplearning4j.nn.graph.*;
import org.deeplearning4j.nn.conf.*;
import org.deeplearning4j.nn.conf.inputs.*;
import org.deeplearning4j.nn.conf.layers.*;
import org.deeplearning4j.nn.weights.*;
import org.deeplearning4j.optimize.listeners.*;
import org.deeplearning4j.datasets.datavec.RecordReaderMultiDataSetIterator;
import org.deeplearning4j.eval.Evaluation;

import org.nd4j.linalg.learning.config.*; // for different updaters like Adam, Nesterovs, etc.
import org.nd4j.linalg.activations.Activation; // defines different activation functions like RELU, SOFTMAX, etc.
import org.nd4j.linalg.lossfunctions.LossFunctions;

In [3]:
import org.nd4j.linalg.dataset.DataSet;
import org.nd4j.linalg.dataset.MiniBatchFileDataSetIterator;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;

In [5]:
INDArray features = Nd4j.rand(new int[]{100, 1, 20, 20}).mul(5);
INDArray labels = Nd4j.rand(100, 1);

In [7]:
features.toString()

[[[[    3.6598,    4.6477,    1.5436  ...    1.0645    0.8515,    1.5316], 
   [    2.2170,    4.5043,    3.7425  ...    0.0231    1.1926,    2.5139], 
   [    3.4177,    1.0760,    1.7559  ...    4.1069    2.9033,    4.4285], 
    ..., 
   [    4.4954,    4.8128,    4.8552  ...    2.5398    2.3941,    3.8950], 
   [    1.3535,    1.7540,    4.0907  ...    1.3173    1.1665,    1.7231], 
   [    1.0145,    2.7807,    4.0424  ...    2.1255    0.5702,    0.3294]]], 


 [[[    1.0119,    1.1183,    1.6285  ...    2.8614    0.8561,    3.1408], 
   [    4.0638,    2.8890,    2.2406  ...    4.3256    1.1436,    3.8390], 
   [    1.8563,    2.9801,    1.9556  ...    3.3519    0.1953,    4.0642], 
    ..., 
   [    0.2446,    3.6767,    0.9542  ...    2.4629    3.9952,    1.8431], 
   [    0.9177,    1.7774,    2.0097  ...    2.6185    1.7257,    4.5988], 
   [    2.3880,    4.7112,    4.5031  ...    1.8687    2.7555,    3.2322]]], 


 [[[    2.1197,    0.1079,    2.5330  ...    3.4760    2.944

In [5]:
features = features.add(features.mean().neg()).div(features.std())

In [6]:
DataSet d = new DataSet(features, labels);

In [7]:
DataSetIterator di = new ListDataSetIterator(d.asList(), 10);

In [8]:
MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
    .seed(123)
    .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
    .list()
    .layer(0, new ConvolutionLayer.Builder().kernelSize(3, 3).stride(1, 1).nIn(1).nOut(20).dropOut(.5).build())
    .layer(1, new OutputLayer.Builder().nOut(1).weightInit(WeightInit.XAVIER).build())
    .setInputType(InputType.convolutional(20, 20, 1))
    .pretrain(false).backprop(true)
    .build()

In [9]:
MultiLayerNetwork network = new MultiLayerNetwork(conf);
network.init();
network.setListeners(new ScoreIterationListener(1));

[IJava-executor-0] INFO org.deeplearning4j.nn.multilayer.MultiLayerNetwork - Starting MultiLayerNetwork with WorkspaceModes set to [training: ENABLED; inference: ENABLED], cacheMode set to [NONE]


In [10]:
for (int i=0; i<3; i++) {
    network.fit(di);
    System.out.println("*** Completed epoch " + i + ".");
}

[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 0 is 0.18465788364410402
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 1 is 0.2753016710281372
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 2 is 0.12080224752426147
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 3 is 0.19649918079376222
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 4 is 0.11572840213775634
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 5 is 0.06093320846557617
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 6 is 0.07324883937835694
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener

*** Completed epoch 0.


[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 10 is 0.03940052688121796
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11 is 0.07562063932418824
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 12 is 0.035659310221672055
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13 is 0.07124021053314208
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 14 is 0.05120176672935486
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15 is 0.026362788677215577
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 16 is 0.03158822357654571
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterati

*** Completed epoch 1.


[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 20 is 0.023371633887290955
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 21 is 0.03858671486377716
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 22 is 0.019564956426620483
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 23 is 0.041256952285766604
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 24 is 0.028369739651679993
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 25 is 0.015617898106575013
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 26 is 0.021478107571601866
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIte

*** Completed epoch 2.


In [8]:
String tmp_dir = System.getProperty("java.io.tmpdir");


/tmp

In [21]:
import org.datavec.image.loader.NativeImageLoader;

In [23]:
NativeImageLoader loader = new NativeImageLoader(28, 28, 3);
INDArray image = loader.asMatrix(new File("/home/bzr0014/Pictures/LSU_HIP.png"));

In [29]:
Arrays.toString(image.shape())

[1, 3, 28, 28]

In [30]:
int seed = 2345;
        double learningRate = 0.05;
        int numInputs = 2;   // x and y.
        int numHiddenNodes = 100;
        int numOutputs = 3 ; //R, G and B value.

        MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
            .seed(seed)
            .weightInit(WeightInit.XAVIER)
            .updater(new Nesterovs(learningRate, 0.9))
            .list()
            .layer(0, new DenseLayer.Builder().nIn(numInputs).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(1, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(2, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(3, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(4, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(5, new OutputLayer.Builder(LossFunctions.LossFunction.L2)
                .activation(Activation.IDENTITY)
                .nIn(numHiddenNodes).nOut(numOutputs).build())
            .build();

        MultiLayerNetwork net = new MultiLayerNetwork(conf);
        net.init();

In [36]:
net.getLayerNames()

[layer0, layer1, layer2, layer3, layer4, layer5]

In [42]:
import org.deeplearning4j.nn.api.Layer

In [45]:
net.getLayer("layer0").paramTable()

{W=[[   -0.1763,   -0.0715,   -0.1460,    0.0425,   -0.1897,    0.0922,    0.0464,   -0.0536,    0.0632,   -0.0231,   -0.0144,    0.1276,   -0.1895,    0.0014,   -0.2097,    0.0362,    0.0952,    0.1855,    0.2081,    0.4102,   -0.0354,    0.0105,   -0.1754,   -0.3234,    0.1678,   -0.0479,   -0.0963,    0.2717,    0.0268,   -0.0288,   -0.4067,    0.0410,    0.0891,    0.1048,   -0.0738,   -0.0762,    0.0015,    0.1213,   -0.1552,   -0.0129,   -0.0070,   -0.1444,    0.0290,   -0.0415,   -0.0335,   -0.1025,    0.0375,   -0.0405,   -0.0173,    0.0072,    0.1228,   -0.0794,   -0.2758,    0.1102,    0.1362,    0.0026,    0.1445,    0.0276,    0.0461,   -0.0537,   -0.2009,   -0.2256,   -0.1905,   -0.0724,    0.1469,   -0.1153,    0.0803,    0.0583,   -0.2742,    0.1661,   -0.1223,   -0.0542,   -0.1534,    0.1126,   -0.2364,   -0.2347,    0.0665,    0.0631,    0.0519,   -0.0588,   -0.0495,   -0.1931,    0.0838,   -0.1271,   -0.2515,   -0.0057,   -0.3719,    0.0924,    0.0603,    0.1344,   -0

In [ ]:
net.getLayers()